In [1]:
# default_exp utils.statistics

In [3]:
# export

import numpy as np
import pandas as pd
import scipy.stats as st
from typing import Callable, Optional, List
from sklearn.preprocessing import MinMaxScaler

In [20]:
# export

#Logging configuration

import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.INFO)

## statistics

> Generic interface for defining distance metrics for multiple purposes (clustering, sample sets comparison, etc).

> Functions for computing divergence metrics (e.g., KL div, Jensen Shannon divergence).

### Utils - Helpers

Estimation for number of bins using <i>Freedman–Diaconis rule</i>

In [38]:
# export

def estimate_bins_number(data: np.ndarray) -> int:
    """Using the Freedman–Diaconis rule"""
    # logging.info(f"Estimating bins number for {data}")
    iqr = get_iqr(data)
    h = (2 * iqr) / (len(data) ** (1/3))
    max_ = data.max()
    min_ = data.min()
    
    n_bins = (max_ - min_) / h
    
    if np.isposinf(n_bins) or np.isneginf(n_bins) or np.isnan(n_bins):
        return 1
    
    return int(np.ceil(n_bins))
    
def get_iqr(data: np.ndarray) -> float:
    q75, q25 = np.percentile(data, [75 ,25])
    iqr = q75 - q25
    return iqr

Bootstrapping-based estimations

In [3]:
# export

def bootstrapping( np_data: np.ndarray, np_func: Callable,
                  n_resamples: Optional[int]=500, sample_size: Optional[int]=None,
                  columns: Optional[List[str]]=['bootstrap_repl'],
                  full_zeros: Optional[bool]=False) -> pd.DataFrame:
    """Estimates a based on bootstrapped samples given data and a function
    For instance, a bootstrap sample of means, or medians. 
    The bootstrap samples are as long as the original set size,
    additionally, bootstrapping is performed based on resampling with replacement (np.random.choice)
    """
    
    if sample_size is None:
        #Creating the boostrapped estimates based on replicates as big as the orignal data size
        sample_size = len(np_data)
        
    #Cleaning NaNs
    np_data_clean = np_data[ np.logical_not( np.isnan(np_data) ) ] 
    
    if full_zeros:
        bootstrap_repl = [ 0.0  for _ in range(n_resamples) ]
    else:
        bootstrap_repl = [ np_func( np.random.choice( np_data_clean, size=sample_size ) ) for _ in range(n_resamples) ]
    
    # Provide info. about empirical and bootstrapped mean
    logging.info("Empirical Mean: " + str(np.mean(np_data_clean)))
    logging.info( "Bootstrapped Mean: " + str( np.mean(bootstrap_repl) ) )
    
    return pd.DataFrame(bootstrap_repl, columns = columns)

In [ ]:
# export

def perform_dim_wise_bootstrapping(data_df: pd.DataFrame,
                                   sample_size: Optional[int]=500,
                                   est_funct: Optional[Callable]=np.mean
                                   ) -> pd.DataFrame:
    """
    Bootstrap dataframe for each dimension in the given df
    
    Parameters
    ---------
    data_df: DataFrame
        dataset containing the data to be bootstrapped
        
    Returns
    -------
    bs_data_df: DataFrame
        Dataset containing boostrapped data
    """
    dims = list(data_df.columns)
    data_dict = {}
    
    for dim in dims:
        dim_data = data_df[dim].values
        zeros_data = verify_if_zeros(dim_data)

        bs_dim_data = bootstrapping(dim_data, np.mean, sample_size=500, full_zeros=zeros_data)
        bs_data = bs_dim_data['bootstrap_repl'].values
        data_dict[dim] = bs_data
        
    return pd.DataFrame(data_dict)

In [ ]:
# export

def confidence_intervals(np_data, conf=0.95):
    """Confidence Intervals with Gaussian Distribution (n>=30)"""
    CI = st.norm.interval(alpha=conf, loc = np.mean( np_data ), scale = st.sem(np_data))
    logging.info( CI )
    return CI

In [6]:
# export

def get_z_confidence_interval(data: np.ndarray, crit_value=2.575):
    """
    Get the z-interval for a given sample set, using the provided critical value
    
    :param data: np.ndarray containing data for estimating the confidence interval
    :param crit_value: float indicating the critical value for the desired confidence
    :return: Tuple[float, float]
             - Containing min and max values of the interval
    """
    mean = np.mean(data)
    std = np.std(data)
    
    int_min = mean - (crit_value * (std/np.sqrt(len(data))))
    int_max = mean + (crit_value * (std/np.sqrt(len(data))))
    
    return int_min, int_max

In [7]:
# export 

def get_t_confidence_interval(data: np.ndarray, crit_value):
    """
    Get the t-interval for a given sample set, using the provided critical value
    
    :param data: np.ndarray containing data for estimating the confidence interval
    :param crit_value: float indicating the critical value for the desired confidence
    :return: Tuple[float, float]
             - Containing min and max values of the interval
    """
    mean = np.mean(data)
    std = np.std(data)
    
    int_min = mean - (crit_value * (std/np.sqrt(len(data))))
    int_max = mean + (crit_value * (std/np.sqrt(len(data))))
    
    return int_min, int_max

In [ ]:
# export

def get_all_t_confidence_intervals(data_df: pd.DataFrame,
                                   crit_value: Optional[float]=2.364
                                   ) -> Dict:
    """
    Get the dimension-wise confidence intervals using t-values
    
    Parameters
    ----------
    data_df: pd.DataFrame
        Data for interval estimation (dimension-wise)
    crit_value: Optional[float]
        default: 2.364
        Crit. value to compute confidence intervals
        
    Returns
    result: Dict
       Dictionary containing tuples with the edge values
       of the intervals
    -------
    """
    result = {}
    for dim in list(data_df.columns):
        min_int, max_int = get_t_confidence_interval(data_df[dim].values, crit_value)
        result[dim] = (min_int, max_int)
    
    return result

In [6]:
# export

def min_max_normalize_df(df: pd.DataFrame):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        
        if max_value == min_value:
            # No need to normalize, all values would be equal
            continue
            
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [19]:
# export 

def min_max_scale_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Perform min-max standarization of the provided dataset
    
    Parameters
    -----------
    df: DataFrame
        Dataset to be standarized
    Returns
    -----------
    Standarized dataset
    """
    # create a scaler object
    scaler = MinMaxScaler()
    scaler.fit(df.values)
    # fit and transform the data
    df_norm = pd.DataFrame(scaler.transform(df.values), columns=df.columns)
    return df_norm

In [ ]:
# export

def min_max_dimwise_scale_df(df: pd.DataFrame) -> pd.DataFrame:
    """
    Perform dimensionwise min-max standarization
    for the provided dataset
    
    Parameters
    -----------
    df: Dataframe
        Dataset to be standarized
    
    Returns
    -----------
    Standarized data frame
    """
    scaler = MinMaxScaler()
    dimensions = list(df.columns)
    
    scaled_data = {}
    for dim in dimensions:
        scaler.fit(df[dim].values)
        scaled_data[dim] = scaler.transform(df[dim].values)
        
    return pd.DataFrame(scaled_data.items(), columns=dimensions)

In [ ]:
# export

def get_t_confidence_interval(data: np.ndarray, crit_value):
    """
    Get the t-interval for a given sample set, using the provided critical value
    Parameters
    ----------
    data: np.ndarray
        Array containing data for estimating the confidence interval
    crit_value: float
        value indicating the critical value for the desired confidence
    
    Returns
    ----------
    Tuple(float, float)
        Edge values for the confidence interval
    """
    mean = np.mean(data)
    std = np.std(data)
    
    int_min = mean - (crit_value * (std/np.sqrt(len(data))))
    int_max = mean + (crit_value * (std/np.sqrt(len(data))))
    
    return int_min, int_max

In [23]:
from nbdev.export import notebook2script
notebook2script()

Converted 0.1_mgmnt.prep.ipynb.
Converted 0.2_mgmnt.prep.files_mgmnt.ipynb.
Converted 0.3_mgmnt.prep.bpe_tokenization.ipynb.
Converted 0.4_mgmnt.prep.tokenization_counting.ipynb.
Converted 0.5_mgmnt.prep.token_mgmnt.ipynb.
Converted 1.1_exp.info.ipynb.
Converted 1.2_exp.desc.metrics.java.ipynb.
Converted 1.4_exp.metrics_python.ipynb.
Converted 1.5_exp.metrics_java.ipynb.
Converted 2.0_repr.codebert.ipynb.
Converted 2.0_repr.i.ipynb.
Converted 2.1_repr.codeberta.ipynb.
Converted 2.1_repr.roberta.train.ipynb.
Converted 2.2_repr.roberta.eval.ipynb.
Converted 2.3_repr.word2vec.train.ipynb.
Converted 2.6_repr.word2vec.eval.ipynb.
Converted 2.7_repr.distmetrics.ipynb.
Converted 2.8_repr.sentence_transformers.ipynb.
Converted 3.1_traceability.unsupervised.eda.ipynb.
Converted 3.2_traceability.unsupervised.approach.d2v.ipynb.
Converted 3.2_traceability.unsupervised.approach.w2v.ipynb.
Converted 4.0_infoxplainer.ir.ipynb.
Converted 4.1_infoxplainer.ir.unsupervised.d2v.ipynb.
Converted 4.2_infox